In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm

from mpmath import besseljzero
from scipy.special import gamma
from scipy.special import jv, iv, ive
from scipy.optimize import differential_evolution
from scipy.interpolate import interp1d

from scipy.stats import pearsonr
from sklearn.metrics import r2_score

import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
def simulate_HSDM_3D(a, mu, eta, ndt, sigma=1, dt=0.001):
    x = np.zeros(mu.shape)
    
    rt = 0
    
    mu_t = np.random.normal(mu, eta) 
    while np.linalg.norm(x, 2) < a(rt):
        x += mu_t*dt + sigma*np.sqrt(dt)*np.random.normal(0, 1, mu.shape)
        rt += dt
    
    theta1 = np.arctan2(np.sqrt(x[2]**2 + x[1]**2), x[0])
    theta2 = np.arctan2(x[2], x[1])   
    
    return rt+ndt, (theta1, theta2)

In [3]:
def k(a, da, t, q, sigma=2):
    return 0.5 * (q - 0.5*sigma - da(t))

def psi(a, da, t, z, tau, q, sigma=2):
    kk = k(a, da, t, q, sigma)
    
    if 2*np.sqrt(a(t)*z)/(sigma*(t-tau))<=700:
        term1 = 1./(sigma*(t - tau)) * np.exp(- (a(t) + z)/(sigma*(t-tau)))
        term2 = (a(t)/z)**(0.5*(q-sigma)/sigma)
        term3 = da(t) - (a(t)/(t-tau)) + kk
        term4 = iv(q/sigma-1, 2*np.sqrt(a(t)*z)/(sigma*(t-tau)))
        term5 = (np.sqrt(a(t)*z)/(t-tau)) * iv(q/sigma, 2*np.sqrt(a(t)*z)/(sigma*(t-tau)))
    else:
        term1 = 1./(sigma*(t - tau))
        term2 = (a(t)/z)**(0.5*(q-sigma)/sigma)
        term3 = da(t) - (a(t)/(t-tau)) + kk
        term4 = ive(q/sigma-1, (a(t) + z)/(sigma*(t-tau)))
        term5 = (np.sqrt(a(t)*z)/(t-tau)) * ive(q/sigma, (a(t) + z)/(sigma*(t-tau)))
    
    return term1 * term2 * (term3 * term4 + term5)

def ie_bessel_fpt(a, da, q, z, sigma=2, dt=0.1, T_max=2):
    g = [0]
    T = [0]
    g.append(-2*psi(a, da, dt, z, 0, q, sigma))
    T.append(dt)
    
    for n in range(2, int(T_max/dt)+2):
        s = -2 * psi(a, da, n*dt, z, 0, q, sigma)

        for j in range(1, n):
            s += 2 * dt * g[j] * psi(a, da, n*dt, a(j*dt), j*dt, q, sigma)

        g.append(s)
        T.append(n*dt)
        
    g = np.asarray(g)
    T = np.asarray(T)
    
    gt = interp1d(T, g)
    return gt

In [4]:
def HSDM_3D_likelihood(prms, RT, Theta):
    a = lambda t: prms[0]**2
    da = lambda t: 0
    ndt = prms[1]
    
    eta = prms[2]
    eta2 = eta**2
    
    mu = np.array([prms[3], prms[4], prms[5]])
    
    fpt = ie_bessel_fpt(a, da, mu.shape[0], 0.000001, 
                        dt=0.01, T_max=max(RT))
    log_lik = 0
    for i in range(len(RT)):
        rt, theta = RT[i], Theta[i]
        if rt - ndt > 0.001 and ndt >= 0:
            x0 = prms[0]*np.cos(theta[0])
            x1 = prms[0]*np.sin(theta[0])*np.cos(theta[1]) 
            x2 = prms[0]*np.sin(theta[0])*np.sin(theta[1])
            
            fixed = 1/(np.sqrt(eta2 * (rt - ndt) + 1))
            
            exponent0 = -0.5*mu[0]**2/eta2 + 0.5*(x0 * eta2 + mu[0])**2 / (eta2 * (eta2 * (rt - ndt) + 1))
            exponent1 = -0.5*mu[1]**2/eta2 + 0.5*(x1 * eta2 + mu[1])**2 / (eta2 * (eta2 * (rt - ndt) + 1))
            exponent2 = -0.5*mu[2]**2/eta2 + 0.5*(x2 * eta2 + mu[2])**2 / (eta2 * (eta2 * (rt - ndt) + 1))
            
            term1 = fixed * np.exp(exponent0)
            term2 = fixed * np.exp(exponent1)
            term3 = fixed * np.exp(exponent2)
            
            density = term1 * term2 * term3 * fpt(rt - ndt)
            
            if 0.1**14 < density:
                log_lik += -np.log(density)
            else:
                log_lik += -np.log(0.1**14)
        else:
            log_lik += -np.log(0.1**14)
        
    return log_lik

In [5]:
recovery_df = {'threshold_true': [],
               'threshold_estimate': [],
               'ndt_true': [],
               'ndt_estimate': [],
               'eta_true': [],
               'eta_estimate':[],
               'mu1_true': [],
               'mu1_estimate': [],
               'mu2_true': [],
               'mu2_estimate': [],
               'mu3_true': [],
               'mu3_estimate': []}

min_threshold = 0.5
max_threshold = 3

min_ndt = 0.1
max_ndt = 1

min_eta = 0.1
max_eta = 1

min_mu = -3
max_mu = 3

In [6]:
for n in tqdm(range(5)):
    threshold = np.random.uniform(min_threshold, max_threshold)
    a = lambda t: threshold
    ndt = np.random.uniform(min_ndt, max_ndt)
    eta = np.random.uniform(min_eta, max_eta)
    mu = np.array([np.random.uniform(min_mu, max_mu), 
                   np.random.uniform(min_mu, max_mu),
                   np.random.uniform(min_mu, max_mu)])
    
    
    recovery_df['threshold_true'].append(threshold)
    recovery_df['ndt_true'].append(ndt)
    recovery_df['eta_true'].append(eta)
    recovery_df['mu1_true'].append(mu[0])
    recovery_df['mu2_true'].append(mu[1])
    recovery_df['mu3_true'].append(mu[2])
    
    RT = []
    Theta = []
    
    for i in range(500):
        rt, theta = simulate_HSDM_3D(a, mu, eta, ndt)
        RT.append(rt)
        Theta.append(theta)
    
    min_ans = differential_evolution(HSDM_3D_likelihood,
                                     args=(RT, Theta), 
                                     bounds=[(min_threshold, max_threshold), 
                                             (min_ndt, max_ndt), (min_eta, max_eta),
                                             (min_mu, max_mu), (min_mu, max_mu), (min_mu, max_mu)])
    
    recovery_df['threshold_estimate'].append(min_ans.x[0])
    recovery_df['ndt_estimate'].append(min_ans.x[1])
    recovery_df['eta_estimate'].append(min_ans.x[2])
    recovery_df['mu1_estimate'].append(min_ans.x[3])
    recovery_df['mu2_estimate'].append(min_ans.x[4])
    recovery_df['mu3_estimate'].append(min_ans.x[5])
    
recovery_df = pd.DataFrame(recovery_df)

100%|████████████████████████████████████████████| 5/5 [18:53<00:00, 226.66s/it]


In [7]:
recovery_df

,threshold_true,threshold_estimate,ndt_true,ndt_estimate,eta_true,eta_estimate,mu1_true,mu1_estimate,mu2_true,mu2_estimate,mu3_true,mu3_estimate
0,2.037209,2.030551,0.897272,0.909960,0.954010,0.972741,2.867305,3.000000,-2.263618,-2.222453,2.033801,2.046148
1,2.883247,2.868284,0.122224,0.137799,0.138129,0.100000,0.082875,0.163523,-1.858587,-1.897551,-1.706674,-1.692792
2,0.889327,0.864112,0.296625,0.300625,0.734895,0.252909,0.411916,0.313622,-0.330908,-0.305607,-2.545393,-2.302281
3,1.729483,1.740170,0.555563,0.560563,0.928115,1.000000,-2.123800,-2.191530,1.553984,1.548678,2.973566,2.990546
4,0.676817,0.673467,0.444595,0.447899,0.237416,0.100000,2.992275,3.000000,0.302785,0.468900,-2.548969,-2.627633


In [9]:
file_name = 'IE_3d_recovery_dvar.csv'
# old_recovery_data = pd.read_csv(file_name, index_col=0)
# recovery_df = pd.concat([old_recovery_data, 
#                          recovery_df]).reset_index(drop=True)
recovery_df.to_csv(file_name)

In [8]:
recovery_df.corr()

,threshold_true,threshold_estimate,ndt_true,ndt_estimate,eta_true,eta_estimate,mu1_true,mu1_estimate,mu2_true,mu2_estimate,mu3_true,mu3_estimate
threshold_true,1.000000,0.999888,-0.095908,-0.078182,-0.113097,0.185221,-0.281369,-0.247040,-0.553050,-0.583593,0.349597,0.344015
threshold_estimate,0.999888,1.000000,-0.088704,-0.071043,-0.108807,0.194251,-0.285082,-0.250670,-0.544404,-0.574930,0.360096,0.354160
ndt_true,-0.095908,-0.088704,1.000000,0.999815,0.722242,0.808553,0.358556,0.365021,-0.084921,-0.071300,0.712347,0.707100
ndt_estimate,-0.078182,-0.071043,0.999815,1.000000,0.718579,0.809612,0.360272,0.367390,-0.100828,-0.087545,0.715599,0.710215
eta_true,-0.113097,-0.108807,0.722242,0.718579,1.000000,0.880229,-0.225143,-0.227793,0.168713,0.156163,0.746625,0.769725
eta_estimate,0.185221,0.194251,0.808553,0.809612,0.880229,1.000000,-0.246918,-0.236011,0.128455,0.114102,0.968397,0.974114
mu1_true,-0.281369,-0.285082,0.358556,0.360272,-0.225143,-0.246918,1.000000,0.999328,-0.520115,-0.478831,-0.342634,-0.359672
mu1_estimate,-0.247040,-0.250670,0.365021,0.367390,-0.227793,-0.236011,0.999328,1.000000,-0.542556,-0.501931,-0.326106,-0.343725
mu2_true,-0.553050,-0.544404,-0.084921,-0.100828,0.168713,0.128455,-0.520115,-0.542556,1.000000,0.998853,0.139664,0.139544
mu2_estimate,-0.583593,-0.574930,-0.071300,-0.087545,0.156163,0.114102,-0.478831,-0.501931,0.998853,1.000000,0.121017,0.119849
